In [0]:
# Given below candidates data:
# | emp_id | experience | salary |
# | 1 | Junior | 10000 |
# | 2 | Junior | 15000 |
# | 3 | Junior | 40000 |
# | 4 | Senior | 16000 |
# | 5 | Senior | 20000 |
# | 6 | Senior | 50000 |

# ❓Hire the candidates who fall under budget of 70000 according to below criteria:
# 📌 First hire Senior within budget
# 📌 Then hire Junior withing remaining budget.

#they have to first fill up the senior position then fill up the junior position
# we will start from senior having least salary as we want to hire maximum person. 
#https://www.youtube.com/watch?v=C9DGxJKBbb4

❓Hire the candidates who fall under budget of 70000 according to below criteria:
📌 First hire Senior within budget
📌 Then hire Junior withing remaining budget

In [0]:
display(my_data)

emp_id,experience,salary
1,Junior,10000
2,Junior,15000
3,Junior,40000
4,Senior,16000
5,Senior,20000
6,Senior,50000


In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [0]:
schema=StructType([
    StructField("emp_id",IntegerType(),True),
    StructField("experience",StringType(),True),
    StructField("salary",IntegerType(),True)
]
)

In [0]:
data=[(1,"Junior",10000),
      (2,"Junior",15000),
      (3,"Junior",40000),
      (4,"Senior",16000),
      (5,"Senior",20000),
      (6,"Senior",50000)]

In [0]:
my_data=spark.createDataFrame(data=data,schema=schema)

In [0]:
display(my_data)

emp_id,experience,salary
1,Junior,10000
2,Junior,15000
3,Junior,40000
4,Senior,16000
5,Senior,20000
6,Senior,50000


In [0]:
from pyspark.sql.functions import sum,col,max,coalesce

In [0]:
from pyspark.sql import Window

In [0]:
# 1. get the running sum with Window function sum
# 𝘀𝘂𝗺("𝘀𝗮𝗹𝗮𝗿𝘆").𝗼𝘃𝗲𝗿(𝗪𝗶𝗻𝗱𝗼𝘄.𝗽𝗮𝗿𝘁𝗶𝘁𝗶𝗼𝗻𝗕𝘆("𝗲𝘅𝗽𝗲𝗿𝗶𝗲𝗻𝗰𝗲").𝗼𝗿𝗱𝗲𝗿𝗕𝘆("𝘀𝗮𝗹𝗮𝗿𝘆"))
running_salary_df=my_data.withColumn("running_salary",sum("salary")
                                     .over(Window.partitionBy("experience")
                                    .orderBy(["salary","emp_id"])))

In [0]:
display(running_salary_df)

emp_id,experience,salary,running_salary
1,Junior,10000,10000
2,Junior,15000,25000
3,Junior,40000,65000
4,Senior,16000,16000
5,Senior,20000,36000
6,Senior,50000,86000


In [0]:
#filter example
# df.filter((col("act_date") >= "2016-10-01") & (col("act_date") <= "2017-04-01"))
# df.filter("act_date >='2016-10-01' AND act_date <='2017-04-01'")
# df.filter(col("act_date").between("2016-10-01", "2017-04-01"))
# df.filter("act_date BETWEEN '2016-10-01' AND '2017-04-01'")


In [0]:
# 2. filter the result with running_salary lower than 70000
# 𝗳𝗶𝗹𝘁𝗲𝗿($"𝗲𝘅𝗽𝗲𝗿𝗶𝗲𝗻𝗰𝗲" === "𝗦𝗲𝗻𝗶𝗼𝗿" && $"𝗿𝘂𝗻𝗻𝗶𝗻𝗴_𝘀𝗮𝗹𝗮𝗿𝘆" < 𝟳𝟬𝟬𝟬𝟬)
senior_df=running_salary_df.filter((col("experience") == 'Senior') & (col("running_salary") < 70000))

In [0]:
display(senior_df)
#This means in 70000 we can hire 2 senior and below re there details.

emp_id,experience,salary,running_salary
4,Senior,16000,16000
5,Senior,20000,36000


In [0]:
remainingbudget_df=senior_df.select(max("running_salary").alias("remain_sal"))

In [0]:
remaining_money=remainingbudget_df.na.fill(value=0).collect()[0][0]
print(remaining_money)

36000


In [0]:
filterjunior_df=running_salary_df.filter((col("experience")=="Junior") & (col("running_salary") < (70000-remaining_money)))
display(filterjunior_df)

emp_id,experience,salary,running_salary
1,Junior,10000,10000
2,Junior,15000,25000


In [0]:
senior_df.unionAll(filterjunior_df).show()

+------+----------+------+--------------+
|emp_id|experience|salary|running_salary|
+------+----------+------+--------------+
|     4|    Senior| 16000|         16000|
|     5|    Senior| 20000|         36000|
|     1|    Junior| 10000|         10000|
|     2|    Junior| 15000|         25000|
+------+----------+------+--------------+



##Test Case 2

In [0]:
testdata2=[(10,"Junior",10000),
      (40,"Junior",10000),
      (20,"senior",15000),
      (30,"Senior",30000),
      (50,"Senior",15000)
      ]

In [0]:
testdata3=[(10,"Junior",15000),
      (40,"Junior",15000),
      (20,"Junior",20000),
      (30,"Senior",80000)
      ]

In [0]:
testdata_df=spark.createDataFrame(data=testdata3,schema=schema)

In [0]:
##calculate running salary
running_salary_df=testdata_df.withColumn("running_salary",sum("salary")
                                     .over(Window.partitionBy("experience")
                                    .orderBy(["salary","emp_id"])))

#first hire senior with salary < 70000  
senior_df=running_salary_df.filter((col("experience") == 'Senior') & (col("running_salary") < 70000))

#Calculate remaining budget
remainingbudget_df=senior_df.select(max("running_salary").alias("remain_sal")) 
remaining_money=remainingbudget_df.na.fill(value=0).collect()[0][0]     

#Now from remainign budget hire the Juniors
filterjunior_df=running_salary_df.filter((col("experience")=="Junior") & 
                                         (col("running_salary") < (70000-remaining_money)))    

#Union senior and Junior
senior_df.unionAll(filterjunior_df).show()                        

+------+----------+------+--------------+
|emp_id|experience|salary|running_salary|
+------+----------+------+--------------+
|    10|    Junior| 15000|         15000|
|    40|    Junior| 15000|         30000|
|    20|    Junior| 20000|         50000|
+------+----------+------+--------------+

